In [ ]:
cd ..

In [ ]:
import json
from importlib import reload
import km.orchestrator.orchestrator as ork
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from random import shuffle

In [ ]:
db_uri = "sqlite:///km.sqlite"
serialized_model_path = "serialized_models/lda_model_50.pkl"
orchestrator = ork.Orchestrator(db_uri=db_uri)
model = orchestrator.load_model(serialized_model_path)
model.explain()

# Smarter labeling of users

In [ ]:
!curl -o notebooks/names.csv https://raw.githubusercontent.com/hadley/data-baby-names/master/baby-names.csv

In [ ]:
kmeans = KMeans(25, n_init=8)

In [ ]:
X = [doc.representation for doc in documents]
labels = kmeans.fit_predict(X)
docs_with_labels = list(zip(documents, labels))

In [ ]:
names = pd.read_csv("notebooks/names.csv")
names = names[names.year == 2008].sort_values("percent", ascending=False)
names = list(names[names.sex == "boy"].iloc[:6].name) + list(names[names.sex == "girl"].iloc[:6].name)

In [ ]:
clusters = list(set(labels))
shuffle(clusters)
users_dict = {}
for i in range(len(names)):
    user_clusters = clusters[(i*4 % 25): (i*4 + 4)%25]
    if i == 6:
        user_clusters = [25, 0, 1, 2]
    user_docs = []
    for cluster in user_clusters:
        cluster_docs = [d.id for d, l in docs_with_labels if l == cluster]
        cluster_size = len(cluster_docs)
        if i < 6:
            user_cluster_docs = cluster_docs[:cluster_size // 2]
        else:
            user_cluster_docs = cluster_docs[cluster_size // 2:]
        user_docs.extend(user_cluster_docs)
    users_dict[i] = {"email": f"{names[i].lower()}@yourcompany.com", "document_ids": user_docs}

In [ ]:
all_docs = [doc for u, info in users_dict.items() for doc in info["document_ids"]]
assert len(set(all_docs)) == len(all_docs)

In [ ]:
json.dump(users_dict, open("../data/news-articles/user_labels.json", "w"), indent=4, sort_keys=True)